# Data 301 Group 11 Project
## Data Loading and Wrangling Functions

## Data Loading

### Import Package Statements

In [47]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### Loading data into pd dataframes

In [75]:
StatCanCPI = []
StatCanBCHPI = []
StatCanCanadaHPI = []
CanadaPIR = []
def loadStatCanCPI():
    StatCanCPI = pd.read_csv('./processed/preprocessed/Stat_Can_CPI_1985_to_Now.csv')
    return StatCanCPI
def loadStatCanBCHPIData():
    StatCanBCHPI = pd.read_csv('./processed/preprocessed/Stat_Can_HPI_BC-only_1986_to_2021_May.csv')
    return StatCanBCHPI
def loadStatCanCanadaHPIData():
    StatCanCanadaHPI = pd.read_csv('./processed/preprocessed/Stat_Can_HPI_Canada_1981_to_2021_May.csv')
    return StatCanCanadaHPI
def loadCanadaPrimeInterestRate():
    CanadaPIR = pd.read_csv('./processed/preprocessed/Canada-Prime-Rate-History.csv')
    return CanadaPIR

In [76]:
StatCanCPI = loadStatCanCPI()
StatCanBCHPI = loadStatCanBCHPIData()
StatCanCanadaHPI = loadStatCanCanadaHPIData()
CanadaPIR = loadCanadaPrimeInterestRate()

In [80]:
CanadaPIR.head()

,Date,Prime Rate,Bank of Canada Overnight Rate
0,Jan-81,18.25,16.89
1,Feb-81,18.25,16.34
2,Mar-81,17.75,17.15
3,Apr-81,18.25,18.81
4,May-81,19.50,18.82
